In [29]:
import socket
import time
import threading

pro_dj_link_begin_bytes = [
  0x51, 0x73, 0x70, 0x74, 0x31, 0x57, 0x6d, 0x4a,
  0x4f, 0x4c
]

In [30]:
broadcast_ip = "192.168.112.255"
interface_ip = "192.168.112.213" # yukikaM3 IP address
interface_mac_address = "3c:18:a0:99:7b:20"

def broadcast_packet(packet_bytes, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.setsockopt(
        socket.SOL_SOCKET, socket.SO_REUSEADDR, 1
    )  # Allow the address/port to be reused instantly
    s.bind((interface_ip, port))
    s.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)  # Enable broadcasting
    s.sendto(bytearray(packet_bytes), (broadcast_ip, port))
    s.close()

# Encode device name to bytes (similar to pro_dj_link_begin_bytes)
# should be 20 bytes (padded with 00)
# Return as list of bytes
def encode_device_name(device_name):
    device_name_bytes = device_name.encode('utf-8')
    device_name_bytes += b'\x00' * (20 - len(device_name_bytes))
    return list(device_name_bytes)

def encode_mac_address(mac_address):
    return list(bytes.fromhex(mac_address.replace(':', '')))

def encode_ip_address(ip_address):
    return list(map(int, ip_address.split('.')))

def print_hex(data):
    print(" ".join("{:02x}".format(c) for c in data))

In [31]:
# Send first-stage channel number claim
# https://djl-analysis.deepsymmetry.org/djl-analysis/startup.html#cdj-assign-stage-1

def send_first_stage(n):
    packet = pro_dj_link_begin_bytes + [0x00, 0x00] + encode_device_name("rekordbox")
    # other stuff + Length of bit idk
    packet += [0x01, 0x03, 0x00, 0x2c]
    # n should go 1-3 on each packet
    packet += [n]
    # thing
    packet += [0x01]
    # mac address
    packet += encode_mac_address(interface_mac_address)

    broadcast_packet(packet, 50000)

# Send 3 times
for i in range(3):
    send_first_stage(i+1)
    time.sleep(0.03)

In [32]:
# Send second-stage channel number claim
# Send first-stage channel number claim
# https://djl-analysis.deepsymmetry.org/djl-analysis/startup.html#cdj-assign-stage-2

def send_second_stage(d, n):
    packet = pro_dj_link_begin_bytes + [0x02, 0x00] + encode_device_name("rekordbox")
    # other stuff + Length of bit idk
    packet += [0x01, 0x03, 0x00, 0x32]
    # ip address
    packet += encode_ip_address(interface_ip)
    # mac address
    packet += encode_mac_address(interface_mac_address)
    # d
    packet += [d]
    # n
    packet += [n]
    # idk
    packet += [0x04, 0x01]

    broadcast_packet(packet, 50000)

# d we need to send
d_needed = [0x11, 0x12, 0x29, 0x2a, 0x2b, 0x2c]

for i in range(6):
    for d in d_needed:
        send_second_stage(d, i+1)
        time.sleep(0.03)

In [33]:
# Keep alive
# https://djl-analysis.deepsymmetry.org/djl-analysis/startup.html#cdj-keep-alive

def send_keep_alive(d):
    packet = pro_dj_link_begin_bytes + [0x06, 0x00] + encode_device_name("rekordbox")
    # other stuff + Length of bit idk
    packet += [0x01, 0x03, 0x00, 0x36]
    # d
    packet += [d]
    # idk
    packet += [0x11]
    # mac address
    packet += encode_mac_address(interface_mac_address)
    # ip address
    packet += encode_ip_address(interface_ip)
    # idk
    packet += [0x01, 0x01, 0x00, 0x00, 0x04, 0x08]

    broadcast_packet(packet, 50000)

# Send keep alive every 2 seconds
while True:
    for d in d_needed:
        send_keep_alive(d)
        time.sleep(2)

KeyboardInterrupt: 